# FB Prophet Model - Wage Rate ($)

Training (1997-2020), Training (2021-2023)

# Packages & Data Loading

In [1]:
# Import packages
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna
from io import StringIO
import requests
import matplotlib.pyplot as plt

from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go
import plotly.io as pio

## Data Loading

In [2]:
# Load data from Google Drive
file_id = '117pk3NKZPfYmHQyP452HeMV7_8Jq6_W0'

# Create the download URL
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Use requests to get file content
response = requests.get(download_url)
response.raise_for_status()  # Ensure no error occurred

# Load the CSV directly into a DataFrame
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)

In [3]:
# Prepare and process data
df_selected = df[['ref_date', 'value_wage']].copy()
df_grouped = df_selected.groupby('ref_date').agg(
    median_wage=('value_wage', 'median')
).reset_index()
df_grouped['ref_date'] = pd.to_datetime(df_grouped['ref_date'])
df_grouped.set_index('ref_date', inplace=True)

# FB Prophet Model

## Assign Training & Testing Sets

In [4]:
# Split into training and testing sets
train_data = df_grouped['1997-01-01':'2020-12-01']
test_data = df_grouped['2021-01-01':'2023-12-01']
train_data = train_data.reset_index().rename(columns={'ref_date': 'ds', 'median_wage': 'y'})
test_data = test_data.reset_index().rename(columns={'ref_date': 'ds', 'median_wage': 'y'})


# Cross Validation

In [5]:
# Optuna optimization
def optimize_prophet(trial):
    # Define hyperparameters
    changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True)
    seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 10.0, log=True)
    holidays_prior_scale = trial.suggest_float('holidays_prior_scale', 0.01, 10.0, log=True)
    seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])

    # Initialize model with suggested parameters
    model = Prophet(
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        holidays_prior_scale=holidays_prior_scale,
        seasonality_mode=seasonality_mode
    )
    model.fit(train_data)

    # Perform cross-validation
    df_cv = cross_validation(model, initial='7300 days', period='365 days', horizon='365 days')
    df_p = performance_metrics(df_cv)
    return df_p['rmse'].mean()

# Model Parameters

In [6]:
# Optimize using Optuna
study = optuna.create_study(direction='minimize')
study.optimize(optimize_prophet, n_trials=50)

# Best parameters
best_params = study.best_params
print("Best Parameters:", best_params)

# Train model with optimized parameters
optimized_model = Prophet(**best_params)
optimized_model.fit(train_data)

[I 2024-11-30 22:30:35,761] A new study created in memory with name: no-name-b116256d-393f-4d71-829a-95ed5322e403
22:30:36 - cmdstanpy - INFO - Chain [1] start processing
22:30:36 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

22:30:36 - cmdstanpy - INFO - Chain [1] start processing
22:30:36 - cmdstanpy - INFO - Chain [1] done processing
22:30:36 - cmdstanpy - INFO - Chain [1] start processing
22:30:36 - cmdstanpy - INFO - Chain [1] done processing
22:30:36 - cmdstanpy - INFO - Chain [1] start processing
22:30:36 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-11-30 22:30:37,011] Trial 0 finished with value: 0.48311363540660285 and parameters: {'changepoint_prior_scale': 0.06981975510339433, 'seasonality_prior_scale': 0.02667753985891395, 'holidays_prior_scale': 1.846334419236635, 'seasonality_mode': 'additive'}. Best is trial 0 with value: 0.48311363540660285.
22:30:37 - cmdstanpy - INFO - Chain [1] start processing
22:30:37 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

22:30:37 - cmdstanpy - INFO - Chain [1] start processing
22:30:37 - cmdstanpy - INFO - Chain [1] done processing
22:30:37 - cmdstanpy - INFO - Chain [1] start processing
22:30:37 - cmdstanpy - INFO - Chain [1] done processing
22:30:37 - cmdstanpy - INFO - Chain [1] start processing
22:30:37 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-11-30 22:30:37,850] Trial 1 finished with value: 0.5322448558599576 and parameters: {'changepoint_prior_scale': 0.0031484955145449165, 'seasonality_prior_scale': 0.05565910284294348, 'holidays_prior_scale': 0.0941060492702703, 'seasonality_mode': 'multiplicative'}. Best is trial 0 with value: 0.48311363540660285.
22:30:37 - cmdstanpy - INFO - Chain [1] start processing
22:30:38 - cmdstanpy - INFO - Chain [1] done processing


...

  0%|          | 0/3 [00:00<?, ?it/s]

22:31:22 - cmdstanpy - INFO - Chain [1] start processing
22:31:22 - cmdstanpy - INFO - Chain [1] done processing
22:31:22 - cmdstanpy - INFO - Chain [1] start processing
22:31:22 - cmdstanpy - INFO - Chain [1] done processing
22:31:22 - cmdstanpy - INFO - Chain [1] start processing
22:31:22 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-11-30 22:31:22,846] Trial 48 finished with value: 0.47477318299395743 and parameters: {'changepoint_prior_scale': 0.49791635014023, 'seasonality_prior_scale': 0.032586922807796834, 'holidays_prior_scale': 0.022350986452727192, 'seasonality_mode': 'additive'}. Best is trial 44 with value: 0.4396953858712598.
22:31:22 - cmdstanpy - INFO - Chain [1] start processing
22:31:23 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

22:31:23 - cmdstanpy - INFO - Chain [1] start processing
22:31:23 - cmdstanpy - INFO - Chain [1] done processing
22:31:23 - cmdstanpy - INFO - Chain [1] start processing
22:31:23 - cmdstanpy - INFO - Chain [1] done processing
22:31:23 - cmdstanpy - INFO - Chain [1] start processing
22:31:23 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-11-30 22:31:23,737] Trial 49 finished with value: 0.5324826508509332 and parameters: {'changepoint_prior_scale': 0.002133715908975894, 'seasonality_prior_scale': 0.06146694276763226, 'holidays_prior_scale': 0.07644704182237474, 'seasonality_mode': 'multiplicative'}. Best is trial 44 with value: 0.4396953858712598.
22:31:23 - cmdstanpy - INFO - Chain [1] start processing
22:31:23 - cmdstanpy - INFO - Chain [1] done processing


Best Parameters: {'changepoint_prior_scale': 0.0010641213815824788, 'seasonality_prior_scale': 0.017571820852098644, 'holidays_prior_scale': 5.935567700550306, 'seasonality_mode': 'additive'}


In [8]:
# Best parameters
best_params = study.best_params
print("Best Parameters:", best_params)


Best Parameters: {'changepoint_prior_scale': 0.0010641213815824788, 'seasonality_prior_scale': 0.017571820852098644, 'holidays_prior_scale': 5.935567700550306, 'seasonality_mode': 'additive'}


# Model Predictions

In [ ]:
# Generate the future dataframe and forecast
future = optimized_model.make_future_dataframe(periods=36, freq='ME')  # Add 36 months to the end of training data
forecast = optimized_model.predict(future)

## Evaluate on test data
test_data['ds'] = test_data['ds'].dt.to_period('M').dt.to_timestamp('M')

# Convert to plain datetime for comparison
forecast_dates = pd.to_datetime(forecast['ds'])
test_dates = pd.to_datetime(test_data['ds'])

# Find common dates manually using set operations
common_dates = list(set(forecast_dates) & set(test_dates))  # Use intersection of sets

# Align forecast and test data on common dates
evaluation = test_data.set_index('ds').loc[common_dates]
evaluation = evaluation.join(forecast.set_index('ds')[['yhat']], how='inner')
evaluation = evaluation.dropna(subset=['y', 'yhat'])

# Model evaluation metrics
actuals = evaluation['y'].values
predictions = evaluation['yhat'].values
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

# Print final model evaluation statistics
print(f"Final Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}")

# Visualizations

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.plot(forecast['ds'], forecast['yhat'], label='Forecasted', linestyle='-', linewidth=2)
plt.scatter(evaluation.index, evaluation['y'], label='Test Actuals', color='red', marker='o')
plt.title("Optimized Prophet Forecast vs Test Set - Median Hourly Wage ($)", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Median Hourly Wage ($)", fontsize=12)
plt.legend(title="Legend", fontsize=10)
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Generate plot
fig = plot_plotly(optimized_model, forecast)

# Add labels and title
fig.update_layout(
    title="Prophet Forecast - Median Hourly Wage ($)",                 
    xaxis_title="Date",                       
    yaxis_title="Median Hourly Wage ($)",        
    legend_title="Legend",                   
    font=dict(size=12),                       
)

# Show plot
fig.show()

In [ ]:
# Generate plot
fig = plot_plotly(optimized_model, forecast)

# Add test data 
fig.add_trace(
    go.Scatter(
        x=test_data['ds'],  # x-axis (test data dates)
        y=test_data['y'],  # y-axis (actual values)
        mode='markers+lines',
        name='True Values (Test Data)',
        line=dict(color='red', dash='dot'),  
        marker=dict(color='red')
    )
)

# Adjust x-axis range (2021-2023)
fig.update_layout(
    xaxis=dict(
        range=['2021-01-01', '2023-12-31'],  # Set x-axis range
        title='Date',  # Add x-axis title
    ),
    yaxis=dict(
        title='Median Hourly Wage ($)',  # Add y-axis title
    ),
    title='Prophet Forecast with Test Data (2021-2023) - Median Hourly Wage ($)',  # Add plot title
)

# Show plot
fig.show()

In [ ]:
# Generate plot
fig = plot_components_plotly(optimized_model, forecast)

# Add title and labels
fig.update_layout(
    title="Prophet Forecast - Median Hourly Wage ($)",     
    font=dict(size=12),                      
)       

# Show plot
fig.show()

# Optional: Download interactive plots (html)

In [ ]:
# 1. Generate the first plot
fig1 = plot_plotly(optimized_model, forecast)
fig1.update_layout(
    title="Prophet Forecast - Median Hourly Wage ($)",
    xaxis_title="Date",
    yaxis_title="Median Hourly Wage ($)",
    legend_title="Legend",
    font=dict(size=12),
)

# Save the first plot as an HTML string
html1 = pio.to_html(fig1, full_html=False)

# 2. Generate the second plot
fig2 = plot_plotly(optimized_model, forecast)
fig2.add_trace(
    go.Scatter(
        x=test_data['ds'],  # x-axis (test data dates)
        y=test_data['y'],  # y-axis (actual values)
        mode='markers+lines',
        name='True Values (Test Data)',
        line=dict(color='red', dash='dot'),
        marker=dict(color='red')
    )
)
fig2.update_layout(
    xaxis=dict(
        range=['2021-01-01', '2023-12-31'],  # Set x-axis range
        title='Date',  # Add x-axis title
    ),
    yaxis=dict(
        title='Median Hourly Wage ($)',  # Add y-axis title
    ),
    title='Prophet Forecast with Test Data (2021-2023) - Median Hourly Wage ($)',
)

# Save the second plot as an HTML string
html2 = pio.to_html(fig2, full_html=False)

# 3. Generate the third plot
fig3 = plot_components_plotly(optimized_model, forecast)
fig3.update_layout(
    title="Prophet Forecast - Components (Median Hourly Wage $)",
    font=dict(size=12),
)

# Save the third plot as an HTML string
html3 = pio.to_html(fig3, full_html=False)

# Combine all HTML strings into one HTML file
combined_html = f"""
<html>
<head>
    <title>Prophet Forecast Plots</title>
</head>
<body>
    <h1>Prophet Forecast - Interactive Plots</h1>
    <h2>1. Full Forecast</h2>
    {html1}
    <h2>2. Forecast with Test Data</h2>
    {html2}
    <h2>3. Forecast Components</h2>
    {html3}
</body>
</html>
"""

# Save the combined HTML to a file
with open("FB Prophet forecast plots (wage).html", "w") as f:
    f.write(combined_html)

# Print a message indicating where the file is saved
print("HTML file saved as 'FB Prophet forecast plots (wage).html'. Open this file to view the interactive plots.")
